# Метрика через callbacks

Давайте вспомним, что как мы создаем и обучаем нейронные
 сети

Небольшой пример- создаем и компилируем

In [ ]:
# Простая Dense сеть
input1 = Input((xTrainScaled.shape[1],))
input2 = Input((xTrainC01.shape[1],))

x1 = Dense(10, activation="relu")(input1)
x2 = Dense(250, activation="relu")(input2)

x = concatenate([x1, x2])

x = Dense(100, activation='relu')(x)
x = Dense(10, activation='relu')(x)
x = Dense(1, activation='linear')(x)

model = Model((input1, input2), x)

model.compile(optimizer=Adam(lr=1e-3), loss='mse',metrics = 'mae')

В процессе обучения оптимизатор стремится минимизировать потери - loss. Алгоритм подсчета этих потерь мы можем выбрать из списка готовых - MAE, MSE, BinaryCrossentropy или придумать свой - уникальный. Алгоритм задается при компиляции модели. За него отвечает параметр loss метода compile.

Процесс обучения можно характеризовать не только поведением функции потерь, но и другими параметрами. Они называются метрики.

Метрики также можно выбирать из списка готовых - MAE, MSE, BinaryCrossentropy, accuracy или писать самому.
Бывает полезным выбирать функцию потерь одного вида, а метрики другого. Например, MAE и MSE. В примере выше так и сделано.

[Создание собственной функции ошибки\метрики](https://colab.research.google.com/drive/1FelcMWA_g8IbICzs4j3w3vf1JfuxymNR?usp=sharing)

Давайте организуем обработку параметров в процессе обучения для нашей задачи. Просто пересчитаем MAE в удобный нам вид. Для этого напишем callback. В процессе  обучении сети возникают события
- начало эпохи
- конец эпохи
- начало батча
- конец батча
и т.д.

Полный список в описании API KERAS, на эти события можно навешивать их обработчики. Это просто функции, которые будут выполняться при наступлении какого-либо события (конец эпохи, например)

Определим наш callback следующим образом - он будет срабатывать по окончании эпохи



In [ ]:
def on_epoch_end(epoch, logs):

  print(epoch, logs)

  pred = model.predict([xTrainScaled[valMask], xTrainC01[valMask]])

  predUnscaled = yScaler.inverse_transform(pred).flatten()

  yTrainUnscaled = yScaler.inverse_transform(yTrainScaled[valMask]).flatten()

  delta = predUnscaled - yTrainUnscaled

  absDelta = abs(delta)

  print("Эпоха", epoch, "модуль ошибки", round(sum(absDelta) / (1e+6 * len(absDelta)),3))

### Коллбэки
pltMae = LambdaCallback(on_epoch_end=on_epoch_end)

включим наш callback в процесс обучения сетки







In [ ]:
history = model.fit([xTrainScaled[~valMask],

                    xTrainC01[~valMask]], yTrainScaled[~valMask],

                    epochs=200,

                    validation_data=([xTrainScaled[valMask], xTrainC01[valMask]],

                    yTrainScaled[valMask]),

                    vrbose=0,

                    callbacks=[pltMae])

Наш callback

**def on_epoch_end(epoch, logs)**

принимает параметры **epoch, logs**

**epoch** - номер эпохи: 0,1,2,3...

**logs** - словарь с параметрами обучения данной эпохи. В нем потери и метрики.

Например:

{'loss': 0.5059476494789124, 'mae': 0.11555033922195435, 'val_loss': 0.7494572997093201, 'val_mae': 0.10996615886688232}

Заметим, что истории всего обучения он не содержит

Этот словарь можно использовать для расчетов и построения графиков после каждой эпохи (в нашем случае).